In [15]:
import os
import sys
import fastf1
try:
    fastf1.Cache.enable_cache(sys.path[0]+"/fastf1_cache") 
except:
    os.makedirs(sys.path[0]+"/fastf1_cache")
    fastf1.Cache.enable_cache(sys.path[0]+"/fastf1_cache") 
from fastf1 import plotting
from matplotlib import pyplot as plt
import matplotlib.cm as cm
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap
from matplotlib.lines import Line2D
import datetime
import seaborn as sns
sns.set_style("darkgrid")
import pandas as pd
import numpy as np
from windrose import WindroseAxes
pd.set_option('display.max_columns', None)

Load Session

In [16]:
year = 2023
location = 'Suzuka'
session = 'Race'

session = fastf1.get_session(year, location, session)
session.load(laps=True, telemetry=True)

core           INFO 	Loading data for Japanese Grand Prix - Race [v3.3.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '4', '81', '16', '44', '55', '63', '14', '31', '10', '40', '22', '24', '27', '20', '23', '2', '18', '11', '77']


In [17]:
laps = session.laps
laps = laps.pick_wo_box()

In [23]:
data = pd.DataFrame()
for _,lap in laps.iterlaps(): 
    data = pd.concat([data, lap.get_telemetry().assign(lapNumber=lap.LapNumber, Driver=lap.Driver)])

core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core      

In [24]:
data.columns


Index(['Date', 'SessionTime', 'DriverAhead', 'DistanceToDriverAhead', 'Time',
       'RPM', 'Speed', 'nGear', 'Throttle', 'Brake', 'DRS', 'Source',
       'Distance', 'RelativeDistance', 'Status', 'X', 'Y', 'Z', 'lapNumber',
       'Driver'],
      dtype='object')

In [25]:
drs_zones = data[data.DRS>10].Distance

In [62]:
drs_start, drs_end = min(drs_zones), max(drs_zones)

In [26]:
import plotly.express as go
fig = go.histogram(x=drs_zones)
fig.show()

Filter data (ONLY DRS)

In [34]:
drs_timings = {}
for driver, data_driver in data.groupby('Driver'):
    drs_timings[driver] = data_driver[((data_driver['Distance']>5500)|(data_driver['Distance']<700))&(data_driver['lapNumber']>0)][['lapNumber', 'SessionTime', 'Distance', 'DRS']]

In [66]:
drs_timings['VER'].loc[1,'SessionTime'].total_seconds()

3783.858

In [68]:
for driver, drs_data in drs_timings.items():
    drs_data.reset_index(drop=True, inplace=True)
    drs_data['shift'] = drs_data['Distance'].diff().fillna(0)
    drs_data['zone'] = np.nan
    zone = 0
    for _, row in drs_data.iterrows():
        if row['shift'] > 1000:
            zone += 1
        drs_data.loc[_,'zone'] = zone
    drs_data['time'] = drs_data['SessionTime'].dt.total_seconds()
    drs_data.drop(columns=['shift', 'SessionTime'])
    drs_timings[driver] = drs_data

In [122]:
zone_duration = {}
drs_on = {}
for driver, samples in drs_timings.items():
    zone_duration[driver] = {}
    drs_on[driver] = []
    for _, zone in samples.groupby('zone'):
        if (zone['DRS']>10).any():
            drs_on[driver].append(_)
        [start, end] = np.interp([drs_start, drs_end], zone['Distance'], zone['time'])
        zone_duration[driver][_] = float(end-start)

In [148]:
def remove_outliers(d):
    a = np.array(list(d.values()))
    outlierConstant = 5
    upper_quartile = np.percentile(a, 75)
    lower_quartile = np.percentile(a, 25)
    IQR = (upper_quartile - lower_quartile) * outlierConstant
    quartileSet = (lower_quartile - IQR, upper_quartile + IQR)
    resultList = []
    for y in a.tolist():
        if y >= quartileSet[0] and y <= quartileSet[1]:
            resultList.append(y)
    return {k:v for k,v in d.items() if v in resultList}

In [149]:
avg_pace = {}
for driver, timings in zone_duration.items():
    avg_pace[driver]={}
    timings = remove_outliers(timings)
    
    avg_pace[driver]['noDRS'] = np.mean([timings[x] for x in timings.keys() if x not in drs_on[driver]])
    avg_pace[driver]['DRS'] = np.mean([timings[x] for x in timings.keys() if x in drs_on[driver]])


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning:

Mean of empty slice.

/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning:

invalid value encountered in scalar divide



In [164]:
import plotly.express as px
y = [time['noDRS']-time['DRS'] for time in avg_pace.values()]
fig = px.bar(x=sorted(avg_pace.keys()), y=y, title='Avg time diff due to DRS', labels={'x':'Driver', 'y':'seconds'})
fig.show()

In [163]:
remove_outliers(zone_duration['ZHO'])

{2.0: 13.480000000000473,
 3.0: 13.266999999999825,
 4.0: 13.105999999999767,
 5.0: 13.039999999999964,
 6.0: 13.394000000000233,
 8.0: 13.159999999999854,
 10.0: 13.23700000000008,
 11.0: 12.922999999999774,
 12.0: 13.11700000000019,
 13.0: 11.936999999999898,
 14.0: 12.880999999999403,
 15.0: 13.11999999999989,
 16.0: 13.261999999999716,
 17.0: 13.0600000000004,
 18.0: 13.11999999999989,
 19.0: 12.780000000000655,
 20.0: 13.079999999999927,
 21.0: 13.199999999999818,
 22.0: 13.079999999999927,
 23.0: 13.240999999999985,
 25.0: 13.279999999999745,
 26.0: 12.621999999999389,
 27.0: 13.079999999999927,
 28.0: 12.894999999999527,
 29.0: 12.88299999999981,
 30.0: 12.79699999999957,
 31.0: 13.100000000000364,
 32.0: 13.079999999999927,
 33.0: 13.024000000000342,
 34.0: 12.856000000000677,
 35.0: 12.800000000000182,
 36.0: 13.133999999998196,
 37.0: 12.593999999999141,
 38.0: 12.972999999999956,
 39.0: 13.040000000000873,
 40.0: 13.251999999998588,
 41.0: 13.09099999999853,
 42.0: 13.039000